In [ ]:
import os
import boto3
import pyspark.sql
from pyspark import SparkContext
from pyspark.sql import SparkSession
import findspark
findspark.init()

# Set-up bucket and spark

In [ ]:
#os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.5.jar pyspark-shell"

Set-up spark session and context.

In [ ]:
spark = SparkSession \
    .builder \
    .appName("computer_vision") \
    .master("local[*]") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable","true") \
    .config ("spark.hadoop.fs.gs.project.id", "project_id") \
    .config("spark.hadoop.fs.gs.auth.service.account.email", "mail") \
    .config("spark.hadoop.fs.gs.auth.service.account.private.key.id", "key_id") \
    .config("spark.hadoop.fs.gs.auth.service.account.private.key", "private_key") \
    .getOrCreate()

In [ ]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

Check bucket content.

In [ ]:
!gsutil ls gs://p8_computer_vision/img/*

Import data from bucket.

In [ ]:
path_img = "gs://p8_computer_vision/img/*"

img_all = spark.read.format("binaryfile").load(path_img)
img_all.printSchema()